# Lab 4: エラー訂正

Dynamic Circuitは、量子コンピューターのエラー訂正にも大きく貢献する可能性があります。もし量子エラー訂正についてご存じなくてもご安心ください。まず古典データでこの概念を紹介し、量子コンピューターに応用していきます。

量子コンピューターでは、物理的な量子ビットがノイズの影響を受けやすいため、エラー訂正が重要です。これは回路を実行する際には常に小さな確率でエラーが発生することを意味します。エラー訂正は、発生したエラーを修正することでこの小さな確率の事象に対処し、最終的に正しい結果を得ることを可能にします。

量子エラー訂正をより学びたい、演習をしたい方向けに、[**Qiskitテキストブックは素晴らしいトピックを用意しています**](https://ja.learn.qiskit.org/course/quantum-hardware/introduction-to-quantum-error-correction-via-the-repetition-code)。

それでは必要なものを用意して始めましょう。

In [ ]:
# Importing all the parts we will need.
from typing import List, Optional

from qiskit import transpile, QuantumCircuit, QuantumRegister, ClassicalRegister, Aer
from qiskit.providers.fake_provider import FakeManilaV2
from qiskit.visualization import plot_histogram


import warnings

warnings.filterwarnings("ignore")

import math

pi = math.pi

## 古典エラー訂正

そもそも、なぜエラー訂正が生まれたのでしょうか？

例えば、電波が悪い時に電話で誰かと話すシーンを考えてみてください。せっかくはっきり話そうとしているのに、メッセージが変わってしまったり、台無しになってしまったりすることがあります。「フレッドとのランチを金曜の予定に入れよう(Let's put lunch with Fred)」が、「金曜にフレッドを殴ろう(Let's punch Fred)」に変わってしまうのです。フレッドの週末は一気に悪くなってしまいました。

メッセージを送るとき、たとえビットのような単純なものであっても、常に小さなエラーが発生する可能性があります。1ビットのデータを有線で送信する場合、ビットが反転してしまい、受信者が「0」の代わりに「1」を受け取るという可能性が少しはあるかもしれません。ケーブルが古く、腐食している場合、このようなことが起こりやすくなります。また、データを保存する際にも、CDに傷がつくのと同じようなことが起こる可能性があります。最悪の場合、プロセッサーで計算中にエラーが発生することもあります。

このようなエラーの可能性は、古典情報の問題点として知られています。

ビットが誤って反転する（AKAエラーが発生する）確率を10％とします。これは、1ビットのデータを送信するごとに、1/10の確率で受信したビットが正しくないことを意味します。`0`のメッセージを10回送信しても、受信したメッセージは「`0001000000`」と表示されるかもしれないのです。問題なのはもうお分かりですね。

最も簡単な解決策の1つは、反復を使うことです。私のビットデータ「`1`」は「`111`」とエンコードされ、同様に「`0`」は「`000`」とエンコードされます。各ビットのデータは、1ビットではなく3ビットでエンコードされます。

これが何の役に立つのでしょう？

もし今、「`000`」を送信してエラーが発生した場合、受信者には「`001`」と表示されるかもしれません。受信者は「`000`」か「`111`」のどちらかが送られてくるはずだと知っているので、おそらく送られたのは「`000`」で、1ビットだけ反転したのだろうと推論することができます。エラーが修正されました！

しかし、複数のエラーが発生した場合はどうでしょう？私たちのメッセージは「`011`」に変わり、受信者は私たちが「`111`」を送るつもりだったと思い込んでしまいます。本当のエラーです！繰り返しを使うことで、そのようなことが起こる可能性を減らすことはできますが、ゼロにするわけではありません。では、具体的にどれくらいの確率でエラーを減らすことができるのでしょうか？調べてみましょう！

以下のコードで、エラーがどのように変化するか、自分で確率を求めることができます。

In [ ]:
# Probability for a 000 message to contain one error:
p1 = 0.10

# Calculating the probability of a message containing 2 or 3 errors:
p3 = 3 * p1 * p1 * (1 - p1) + p1 * p1 * p1

print("Percent chance the 000 message has one error: {}".format(p1 * 100))
print("Percent chance the 000 message has 2 or 3 errors: {:.4f}".format(p3 * 100))

反復を用いることで、メッセージを台無しにするエラーの可能性を2.8％にまで減らすことができます（2ビット以上が反転した場合）。これは、1ビットしか使わない場合の10％よりもずっと良い結果です。しかし、これは10％という作り出したエラーに過ぎません。あなたもエラー率を設定できます。試してみてください。エラーの割合`p1`を変えて、2つ以上のエラーが発生する確率にどのような影響を与えるか見てみましょう。

繰り返しがどのように役立つかを知っている私たちは、上のメッセージを「エンコード」する簡単なプログラムを実装することができます。1ビットを3ビットに変換するエンコードです。そして、3ビットを1ビットに戻す、つまり「デコード」するプログラムも作れます。実際、始めに取り組むexerciseとしては良さそうです。

### Exercise 1

入門編として、量子ビット0、1、2を使った3ビットの状態を、以下のルールに従って量子ビット4の1ビットにデコードする「デコーダー」を作ってみましょう。

- 000->0
- 001->0
- 010->0
- 100->0
- 111->1
- 110->1
- 101->1
- 011->1

_ヒント: ビットを反転させるだけなので、位相を導入するようなゲートは必要ありません。必要なのは[**基本ゲート**](https://qiskit.org/documentation/apidoc/circuit_library.html#standard-gates)だけです。_

In [ ]:
# Creating a simple decoder for the classical case
def create_decoder(qr: QuantumRegister, cr: ClassicalRegister) -> QuantumCircuit:
    # Expect a bit being encoded in the first 3 qubits and decode it into the 4th qubit
    # Make sure values of the first 3 qubit stays the same

    qc = QuantumCircuit(qr, cr)
    q0, q1, q2, q3 = qr
    (c0,) = cr

    ####### your code goes here #######

    return qc

では、1をエンコードした場合のデコードの様子を確認してみましょう。

いろいろな入力（エラーも含む）で遊んでみて、エラー訂正回路がどのような動作をするのか試してみましょう。

In [ ]:
# We expect a bit being encoded in the first 3 qubits and decode it into the 4th qubit
qr = QuantumRegister(4)
cr = ClassicalRegister(1)

q0, q1, q2, q3 = qr

# To encode a 1. Change them to test the other encodings.
encoder = QuantumCircuit(qr, cr)
encoder.x(q0)
encoder.x(q1)
encoder.x(q2)

decoder = create_decoder(qr, cr)
qc1 = encoder.compose(decoder)

qc1.draw("mpl")

デコーダーが正しく動作していると思ったら、以下のコードでテストしてください。

In [ ]:
# Submit your circuit
from qc_grader.challenges.spring_2023 import grade_ex4a

grade_ex4a(create_decoder(qr, cr))

# 量子の場合

## 量子の場合は何が複雑なのか

古典の場合、このような誤り訂正符号を持つことは、ビットを数回繰り返すだけなので比較的簡単です。

もちろん、もっと複雑な符号も存在しますが、最終的にはすべての符号が情報を繰り返すことによってある種の冗長性を持つことになります。

しかし量子の場合、以下の2つの理由からそれを実現することができません。

1. 複製不可能定理から量子ビットをコピーすることはできない。

2. 量子ビットを測定すると、その状態が崩れてしまうのでもつれた量子ビットを扱うときは注意が必要。

これは少しがっかりするような話ですが、それでもエラー訂正は可能です。

しかし、情報を保存するために、いくつかの量子ビットが必要なだけでなく（上で説明したような反復）、いわゆる**スタビライザー**として使用する、いくつかの追加のアンシラ（補助）量子ビットが必要です。

補助量子ビット自体は、状態を保存する量子ビットとエンタングルしないという考え方ですが、測定時に起こりうるエラーについてヒントを与えてくれます。

2つの量子ビットを使用します。エンコードに使用するものと、スタビライザーに使用するものです。

# ビット反転反復符号の実装

エラー訂正の実装から始められるように、すべてを準備しましょう。

In [ ]:
# Setup a base quantum circuit for our experiments
encoding = QuantumRegister(3)
stabilizer = QuantumRegister(2)

encoding_q0, encoding_q1, encoding_q2 = encoding
stabilizer_q0, stabilizer_q1 = stabilizer

# Results of the encoding
results = ClassicalRegister(3)

result_b0, result_b1, result_b2 = results

# For measuring the stabilizer
syndrome = ClassicalRegister(2)

syndrome_b0, syndrome_b1 = syndrome

# The actual qubit which is encoded
state = encoding[0]

# The ancillas used for encoding the state
ancillas = encoding[1:]


# Initializing
def initialize_circuit() -> QuantumCircuit:
    return QuantumCircuit(encoding, stabilizer, results, syndrome)

## 量子ビットの初期化

量子状態をエラーから守るために、最初に備えておく必要があります。
一般に状態$$|\Psi_0\rangle \rightarrow |\Psi_1\rangle = (\alpha |0\rangle + \beta |1\rangle)$$を作り出すことが可能です。
以下の回路で物理的な状態$$|\Psi_1\rangle = |1\rangle$$を準備します。

In [ ]:
initial_state = initialize_circuit()

initial_state.x(encoding[0])

initial_state.draw(output="mpl")

## 量子ビットのエンコード

古典の場合と同様に初期化された量子ビットを保存するために繰り返しを用いたくなります。

やりたいことは、状態$|\Psi_1\rangle = (\alpha |0\rangle + \beta |1\rangle)$から状態$ (\alpha |000\rangle + \beta |111\rangle)$へのマッピングをエンコーディング$U_{en}|\Psi_0\rangle$で実行することです。

この状態はエンタングル状態で、1つの量子ビットが測定されると他の2つの量子ビットの出力も確定します。

故に、初めの1量子ビット状態から3量子ビットのエンタングル状態を作るためにCXゲートを使います。

In [ ]:
# Encoding using bit flip code
def encode_bit_flip(qc, state, ancillas):
    qc.barrier(state, *ancillas)
    for ancilla in ancillas:
        qc.cx(state, ancilla)
    return qc


# The circuit encoding our qubit
encoding_circuit = encode_bit_flip(initialize_circuit(), state, ancillas)

# The circuit including all parts so far
complete_circuit = initial_state.compose(encoding_circuit)
complete_circuit.draw(output="mpl")

## デコード回路の準備

元々の状態へデコードするために、逆の操作を実行するデコーダーを構築しなければなりません。つまり$ (\alpha |000\rangle + \beta |111\rangle)$を$|\Psi_1\rangle = (\alpha |0\rangle + \beta |1\rangle)$ へマッピングするデコーダー$U_{de}|\Psi_0\rangle$です。

全く逆の操作なので、エンコーダーを逆転させればよいです。

$U_{de} = U_{en}^\dagger$

In [ ]:
# Decoding (doing the reverse)
def decode_bit_flip(qc, state, ancillas):
    qc.barrier(state, *ancillas)
    for ancilla in ancillas:
        qc.cx(state, ancilla)
    return qc


decoding_circuit = decode_bit_flip(initialize_circuit(), state, ancillas)

decoding_circuit.draw(output="mpl")

## スタビライザーの測定

$A$をターゲット、$B$をソースとするCXゲートを用いることで、量子ビット$A$を別の量子ビット$B$とエンタングルさせることができることを先ほど確認しました（$B$がすでに重ね合わせやエンタングル状態にある場合）。

その後、$A$をターゲットとして、$B$（あるいは、$A$と完全にエンタングル、相関する別の量子ビット）をソースとする別のCXゲートを使うことで、再びエンタングルメントを解くことができます。

スタビライザーを測定して、発生した潜在的なエラーを把握したいので、スタビライザーが状態をエンコードする量子ビットとエンタングルしていないことが重要です。

この事実を知れば、各スタビライザーに偶数個のCXゲートを適用する必要があることは明らかです。スタビライザーが有用であるためには、ビット反転エラーが発生したかどうか、そして3つのエンコーディング量子ビットのうちどの量子ビットで発生したかを測定する必要があります。

### Exercise 2

単一ビット反転エラーを検出する測定ができるように、シンドロームビットを計算してください。シンドロームビットを測定し、スタビライザー量子ビットを`0`状態にリセットするコードを用意しました。

さまざまな方法があるので、できるだけ単純な符号化でスタビライザーを手に入れましょう。

`00` -> エラーなし 

`01` -> 量子ビット0（1番目の量子ビット）にエラーあり

`10` -> 量子ビット1（2番目の量子ビット）にエラーあり

`11` -> 量子ビット2（3番目の量子ビット）にエラーあり

*ヒント：3つのエンコーディング量子ビットは完全に相関しているはずですが、もしそうでない場合、そのうちの1つにビット反転エラーが発生しています。*


In [ ]:
# Add functions such that the classical bits can be used to see which qubit is flipped in the case a single qubit is flipped.
# Use 2 classical bits for it.
# 0 = 00 = no qubit flipped
# 1 = 01 = first qubit (qubit 0) flipped
# 2 = 10 second qubit (qubit 1) flipped
# 3 = 11 = third qubit (qubit 2) flipped
def measure_syndrome_bit(qc, encoding, stabilizer):
    qc.barrier()
    encoding_q0, encoding_q1, encoding_q2 = encoding
    stabilizer_q0, stabilizer_q1 = stabilizer

    ####### your code goes here #######

    ####### don't edit the code below #######
    qc.barrier()
    qc.measure(stabilizer, syndrome)
    with qc.if_test((syndrome_b0, 1)):
        qc.x(stabilizer_q0)
    with qc.if_test((syndrome_b1, 1)):
        qc.x(stabilizer_q1)

    return qc


syndrome_circuit = measure_syndrome_bit(initialize_circuit(), encoding, stabilizer)

complete_circuit = initial_state.compose(encoding_circuit).compose(syndrome_circuit)
complete_circuit.draw("mpl")

In [ ]:
# Submit your circuit
from qc_grader.challenges.spring_2023 import grade_ex4b

grade_ex4b(complete_circuit)

## エラー訂正

これでスタビライザーを構築し、それを測定することでエラーシンドロームを取得できるようになりました。もちろん、エラーが発生したことを示すだけでなく、エラーの修正もできるようにしたいです。

そこでDynamic Circuitを使い、シンドロームの測定値を用いて、潜在的なエラーを修正することにします。古典のケースと同様に、最大でも1つのエラーしか修正できません。それ以上のエラーを修正したい場合は、5エンコード量子ビット以上の長いコードが必要です。

上記のようにうまい方法のシンドローム測定を選んだので、1つのエラーの場合、どの量子ビットが反転しているかが正確に分かるので、エラーを修正するのは非常に簡単です。

### Exercise 3

測定されたシンドロームに応じてエラーを修正してください。

In [ ]:
# Correct the errors, remember how we encoded the errors above!
def apply_correction_bit(qc, encoding, syndrome):
    qc.barrier()
    encoding_q0, encoding_q1, encoding_q2 = encoding

    ####### your code goes here #######

    return qc


correction_circuit = apply_correction_bit(initialize_circuit(), encoding, syndrome)
complete_circuit = (
    initial_state.compose(encoding_circuit)
    .compose(syndrome_circuit)
    .compose(correction_circuit)
)
complete_circuit.draw(output="mpl")

あとは、エンコード量子ビットを測定するのみです。測定前にデコーダー回路を適用して、初期状態を復元します。

もし、何もエラーがなく完璧に動作するのであれば、最初の量子ビットを測定するだけでよいのですが、必ずしもそうではないので、すべての量子ビットを測定して、何か悪いことが起きていないかどうかを確認します。

In [ ]:
def apply_final_readout(qc, encoding, results):
    qc.barrier(encoding)
    qc.measure(encoding, results)
    return qc


measuring_circuit = apply_final_readout(initialize_circuit(), encoding, results)
complete_circuit = (
    initial_state.compose(encoding_circuit)
    .compose(syndrome_circuit)
    .compose(correction_circuit)
    .compose(decoding_circuit)
    .compose(measuring_circuit)
)
complete_circuit.draw(output="mpl")

In [ ]:
# Submit your circuit
from qc_grader.challenges.spring_2023 import grade_ex4c

grade_ex4c(complete_circuit)

## テストの準備
 
これですべてが揃ったので、正しい出力が得られるかどうかをテストすることができます。

実装が正しいことを確認するために、エラーなしで最初のテストを行います。

In [ ]:
# We first choose a simulator as our backend without any noise
backend = Aer.get_backend("qasm_simulator")
# We run the simulation and get the counts
counts = backend.run(complete_circuit, shots=1000).result().get_counts()
# And now we plot a histogram to see the possible outcomes and corresponding probabilities
plot_histogram(counts)

正しい結果が得られていることがわかります（初期量子ビットを1の状態で初期化したので、`00 001`が得られるはずです）。

エンコードに使用した他の量子ビットは、予想通り処理後`0`の状態になっているのがわかりますね。


さて、この回路がノイズなしで動作することがわかったので、ノイズを追加してみましょう！

そのために、ノイズを含むibm_manilaバックエンドをシミュレートするシミュレータを使用します：

*注：このバックエンドはレイアウトが単純なので選びました。レイアウトについては、後ほど詳しく見ていきます。

In [ ]:
# We choose a simulator for Manila a 5 qubit device, includes errors
backend = FakeManilaV2()
# We run the simulation and get the counts
counts = backend.run(complete_circuit, shots=1000).result().get_counts()
# We plot a histogram to see the possible outcomes and corresponding probabilities
plot_histogram(counts)

間違った結果も出ますが、全体としてはほとんどの結果が正しいです。これは良い兆候であり、ノイズがあっても我々のコードが動作することを意味します。

しかし、エラー訂正なしの場合との比較がないのでこのスキームの良さはわかりません。なのでエラー訂正のステップがない場合どの程度の性能があるのかを見てみましょう：

In [ ]:
qc3 = (
    initial_state.compose(encoding_circuit)
    .compose(syndrome_circuit)
    .compose(decoding_circuit)
    .compose(measuring_circuit)
)


# We choose a again FakeManila
backend = FakeManilaV2()
# let the simulation run
counts = backend.run(qc3, shots=1000).result().get_counts()
# and plot our results
plot_histogram(counts)

結果はほぼ同じか、あるいは若干悪くなっていることがわかります。これはエンコード量子ビットを作成後、使用していないためです。

これらの量子ビットを使って計算をすると、通常は何らかの誤差が生じますが、今回はそのようなことはありません。

一方、エラー訂正の部分は、時間のかかる演算を行うため、エラーが発生する可能性があります。

テスト目的に、制御された方法でエラーを発生させる回路を作成します：

- 修正対象であるビット反転エラーを導入したい。

- 3エンコード量子ビットのエラーが互いに独立であるようにしたい。

- エラーが発生する確率を選択できるようにしたい。

- 入力はパーセンテージで、出力はその確率でエラーを発生させる回路にしたい。

### Exercise 4

上で定義したノイズを追加する回路を構築してください。

In [ ]:
# Add some errors as defined above (only add errors to the encoding qubits)
def make_some_noise(qc, encoding, syndrome, error_percentage):
    encoding_q0, encoding_q1, encoding_q2 = encoding
    syndrome_b0, syndrome_b1 = syndrome

    ####### your code goes here #######

    return qc


# Constructing a circuit with 10% error rate (for each of the encoding qubit)
noise_circuit = make_some_noise(initialize_circuit(), encoding, syndrome, 10)
noise_circuit.draw(output="mpl")

以下のコードを使用して、10%のエラー率を導入する回路を作成し、関数をテストしてください。

In [ ]:
# Submit your circuit
from qc_grader.challenges.spring_2023 import grade_ex4d

grade_ex4d(noise_circuit)

10％の確率でエラーを導入した場合、エラー訂正がどれだけうまく動くか検証します。

In [ ]:
qc4 = (
    initial_state.compose(encoding_circuit)
    .compose(noise_circuit)
    .compose(syndrome_circuit)
    .compose(correction_circuit)
    .compose(decoding_circuit)
    .compose(measuring_circuit)
)


# We choose again FakeManila
backend = FakeManilaV2()
# let the simulation run
counts = backend.run(qc4, shots=1000).result().get_counts()
# and plot our results
plot_histogram(counts)

結果は悪化してしまいましたが、ほとんどの場合、`001`が取得出来ていることが分かります。

これで、最初の誤り訂正符号を作成し、テストすることに成功しました。


もし位相エラーも補正したいのであれば、別の方式とより多くの量子ビットが必要です。

エラー訂正についてもっと知りたい方は、こちらもご覧ください： 

https://learn.qiskit.org/course/ch-labs/lab-10-quantum-error-correction

# Short Outlook: エラー訂正とハードウェアレイアウト

上記では、すでにシミュレートされたハードウェアのシミュレーターを使用しましたが、実機でより複雑なものを実行したい場合は、レイアウトも考慮する必要があります。


デバイスのレイアウトは重要です。理想的なケースのようにすべての量子ビットが互いに接続されているわけではないので、レイアウトを考えなければ、3つのcx演算からなるスワップ演算がたくさん使われることになります。もちろん、エラーも発生します。



実機で次のような量子ビットの並びがあるとします。

    0 - 1 - 2 - 3 - 4 

この物理量子ビットを、エラー訂正回路で使用した論理量子ビットにどのようにマッピングするのでしょうか？

ここでは、量子ビットの最初の接続（エンタングル）は、エラー訂正部分よりも「簡単」だと仮定します。エラー訂正部分は何度か繰り返したいので、エラー訂正部分のみを考えて、これらの量子ビットをどのようにマッピングするのでしょうか？



以下のコードで、レイアウトの違いで異なる回路になることを確認できます。

In [ ]:
from qiskit.circuit import IfElseOp

# Fill in a better layout to test!!
initial_layout = [0, 1, 2, 3, 4]

# We use Manila as our potential backend, since it has the wanted layout
backend = FakeManilaV2()

# Temporary workaround for fake backends. For real backends this is not required.
backend.target.add_instruction(IfElseOp, name="if_else")

# And now we transpile the circuit for the backend.
qc_transpiled = transpile(complete_circuit, backend, initial_layout=initial_layout)

qc_transpiled.draw()

完璧なレイアウトはありません。特に、3つの量子ビットをエンタングルさせるために、最初のうちはエラー訂正とは異なる接続が必要なためです。


[0, 4, 2, 1, 3]というレイアウトは、訂正に必要なすべてのCXゲートを直接接続することができ、その部分は何度も実行される可能性があるため、理にかなっています。


また、回路を構成するために最初にスタビライザーとして使用される量子ビットを使用することができるので、これらの量子ビットで初期設定（エンタングル）をどのようにうまく行うことができるかを考える余地もあります。



場合によってかなり異なることを示すために、次のレイアウトを見てみましょう。どのように量子ビットをマッピングするのでしょうか？


    2 - 3 - 4 
            |
           15
            |
           22

上記と同様に[2, 4, 22, 3, 15]のレイアウトが理にかなっていると言えるでしょう。必要なCXゲートをすべて直接接続することができるからです。

どちらのレイアウトも単純な線であるため、これらのレイアウト間に差異はないことがわかります。しかし、IBM_Quitoのバックエンドを見ると、他の5量子ビットデバイスもあることがわかります。Manilaを選んだのは、exerciseに適していたからです。


これらの小さな例は、実際のハードウェアでは、量子ビットのレイアウトに注意しなければならないことを示すものです！

**このことはbonus exerciseで特に重要になります！**。